In [251]:
from utils.dataloader  import load_data
import torch
from utils.confLoader import *
import matplotlib.pyplot as plt
import torch.nn as nn
import qsimcirq
import time
import cirq
from math import pi
import torch.optim as optim
import torch.nn.functional as F

In [83]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available. Training on CPU...')
    device = torch.device('cpu')
else:
    print('CUDA is available. Training on GPU...')
    device = torch.device('cuda:0')

CUDA is available. Training on GPU...


In [84]:
loader = load_data(train_labels_path, test_labels_path, train_image_path, test_image_path, columns, itype = '.jpg', batch_size = 4, shuffle=True, do_random_crop = False, device = 'cpu')
train_loader, test_loader, valid_loader = loader.create_loader()

In [85]:
def display(image_tensor, detach=False):
    # Assuming image_tensor is of shape [1, 3, 224, 224]
    assert len(image_tensor.shape) == 4, "Invalid input shape"
    if detach:
        image_np = image_tensor.detach().numpy()[0]  # Convert to NumPy array and remove singleton dimensions
    else:
        image_np = image_tensor.squeeze().numpy()

    num_layers = image_np.shape[0]  # Number of layers (channels)
    num_rows = (num_layers + 2) // 3  # Calculate the number of rows needed

    # Create a grid of subplots for each layer
    fig, axes = plt.subplots(num_rows, 3, figsize=(15, 5 * num_rows))

    # Flatten axes if we have just one row
    if num_rows == 1:
        axes = axes.reshape(1, -1)

    # Display each layer as a subplot
    for i in range(num_layers):
        row = i // 3
        col = i % 3
        axes[row, col].imshow(image_np[i], cmap="gray")
        axes[row, col].set_title(f"Layer {i + 1}")
        axes[row, col].axis('off')  # Turn off axis ticks and labels

    plt.tight_layout()
    plt.show()

In [215]:
def loading_bar( current_value, total_value, bar_length=40):
    progress = min(1.0, current_value / total_value)
    arrow = '■' * int(progress * bar_length)
    spaces = ' ' * (bar_length - len(arrow))
    print(f'\r[{arrow}{spaces}] {int(progress * 100)}%', end='', flush=True)

In [94]:
def kernel3(P, weight):
    circuit = cirq.Circuit()
    # bias addition kernel
    Q = [cirq.GridQubit(i,0) for i in range(4)]
    W = cirq.GridQubit(0,1)
    keys = ["q"]
    for i in range(4):
        circuit.append(cirq.ry(P[i] * pi).on(Q[i]))
    
    for i in range(4):
        circuit.append(cirq.rx(weight[i]*pi).on(W))
        circuit.append(cirq.CNOT(W, Q[i]))

    circuit.append(cirq.measure(Q[3], key=keys[0]))
    
    return circuit, keys

In [284]:
# from models.circuits.ConvKernel import kernel2, kernel3


class Quanv2D(nn.Module):
    def __init__(
        self,
        simulator,
        in_channels=1,
        out_channels=1,
        kernel_size=2,
        stride=1,
        precision=10,
    ):
        super().__init__()
        self.simulator = simulator
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.kernel = kernel3
        self.precision = precision
        self.extension_factor = self.out_channels // self.in_channels
        self.weight = nn.Parameter(
            torch.rand((out_channels, self.kernel_size * self.kernel_size)),
            requires_grad=True,
        )

    def __repr__(self):
        return f"Quanv2D( {self.in_channels}, {self.out_channels}, kernel =({self.kernel_size},{self.kernel_size}) "\
            f"stride ={self.stride, self.stride}, precision={self.precision} )"

    def patches_generator(self, image):
        image_height, image_width = image.size()
        output_height = (image_height - self.kernel_size) // self.stride + 1
        output_width = (image_width - self.kernel_size) // self.stride + 1

        for h in range(output_height):
            for w in range(output_width):
                patch = image[
                    h * self.stride : (h * self.stride + self.kernel_size),
                    w * self.stride : (w * self.stride + self.kernel_size),
                ]
                patch = patch.reshape(self.kernel_size * self.kernel_size).tolist()
                yield patch, h, w

    def forward(self, x):
        self.inputs = x
        print(x.requires_grad)
        kernel_height, kernel_width = (2, 2)
        count, in_channels, image_height, image_width = x.size()
        expansion = self.in_channels * 4 == self.out_channels
        output_height = (image_height - kernel_height) // self.stride + 1
        output_weidth = (image_width - kernel_width) // self.stride + 1
        result = torch.zeros(count, self.out_channels, output_height, output_weidth)
        for c in range(count):
            for ic in range(in_channels):
                for patch, h, w in self.patches_generator(x[c][ic]):
                    for t in range(self.extension_factor):
                        W = [
                            w.item()
                            for w in self.weight[ic * self.extension_factor + t]
                        ]
                        circuit, keys = self.kernel(patch, W)
                        res = self.simulator.run(circuit, repetitions=self.precision)
                        result[c][ic * self.extension_factor + t][h][w] = (
                            res.histogram(key="q")[1] * 0.1
                        )
        result.requires_grad = True
        return result

    def backward(self, grad_output):
        print("Running backward function", print(grad_output.shape))
        grad_input = torch.zeros_like(self.inputs)
        grad_weight = torch.zeros_like(self.weight)
        error = 9/0
        return self.inputs, self.weight
        # for c in range(grad_output.size(0)):
        #     for ic in range(self.in_channels):
        #         for patch, h, w in self.patches_generator(self.inputs[c][ic]):
        #             for t in range(self.extension_factor):
        #                 W = [w.item() for w in self.weight[ic * self.extension_factor + t]]
        #                 circuit, keys = self.kernel(patch, W)
        #                 res = self.simulator.run(circuit, repetitions=self.precision)

        #                 # Compute gradients for input
        #                 grad_input_patch = grad_output[c][ic * self.extension_factor + t][h][w] * res.gradient(key="q")[1] * 0.1
        #                 grad_input[c][ic][h * self.stride:(h * self.stride + self.kernel_size), w * self.stride:(w * self.stride + self.kernel_size)] += grad_input_patch

        #                 # Compute gradients for weight
        #                 # You need to check if the output of res.gradient(key="w") is correct
        #                 grad_weight[ic * self.extension_factor + t] += grad_output[c][ic * self.extension_factor + t][h][w] * res.gradient(key="w")[t].tolist()

In [292]:
class QuantamModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.simulator = qsimcirq.QSimSimulator()
        self.conv1 = nn.Conv2d( in_channels = 3, out_channels = 12, kernel_size = 2, stride = 1 )
        self.conv2 = nn.Conv2d( in_channels = 12, out_channels = 48, kernel_size = 2, stride = 1 )
        self.conv3 = nn.Conv2d( in_channels= 48, out_channels= 48, kernel_size= 2, stride= 2)
        self.convq = Quanv2D(self.simulator, in_channels = 48, out_channels = 48, kernel_size = 2, stride = 1)
        self.convq2 = Quanv2D(self.simulator, in_channels = 48, out_channels = 96, kernel_size = 2, stride = 1)
        self.fc1 = nn.Linear(96*12*12, 50)
        self.fc2 = nn.Linear(50, 5)
        self.maxpool = nn.MaxPool2d(2, stride = 2)
        self.minpool = nn.AvgPool2d(2, stride = 2)
        self.activation = nn.SiLU()
        self.activation2 = nn.ReLU()
        
    def forward(self, x):
        # print(x.size())
        x = self.activation(self.maxpool(self.conv1(x))) # 12x111x111
        # print(x.size())
        x = torch.relu(self.maxpool(self.conv2(x))) # 48x55x55
        # print(x.size())
        x = self.activation(self.conv3(x)) # 48x27x27
        # print(x.size())
        x = self.activation2(self.maxpool(self.convq(x))) # 48x13x13
        x = self.activation(self.convq2(x)) # 48x4x4
        # print(x.size())
        x = x.view(-1, 96*12*12)
        # print(x.size())
        x = torch.relu(self.fc1(x))
        # print(x.size())
        x = torch.relu(self.fc2(x))
        # print(x.size())
        return x

In [293]:

simulator = qsimcirq.QSimSimulator()
model = QuantamModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [294]:
print(model)

QuantamModel(
  (conv1): Conv2d(3, 12, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(12, 48, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(48, 48, kernel_size=(2, 2), stride=(2, 2))
  (convq): Quanv2D( 48, 48, kernel =(2,2) stride =(1, 1), precision=10 )
  (convq2): Quanv2D( 48, 96, kernel =(2,2) stride =(1, 1), precision=10 )
  (fc1): Linear(in_features=13824, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=5, bias=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (minpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (activation): SiLU()
  (activation2): ReLU()
)


In [287]:
total_loss = 0
start_time = time.time()
for epoch in range(1):
    print(f"Running Epoch no: {epoch+1}")
    for i, data in enumerate(test_loader):
        torch.autograd.set_detect_anomaly(True)
        inputs, labels = data['image'], data['label']
        print(inputs.shape, labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(inputs.size(), outputs.size(), labels.size())
        print(outputs)
        loss = criterion(outputs, labels)
        print("loss :",loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # loading_bar(i, 24)
        break
delta = time.time() - start_time
print(f"Finished training in {delta:0.6f} sec")
print("Total Loss : ", total_loss)

Running Epoch no: 1
torch.Size([1, 3, 224, 224]) tensor([4])
True
True
tensor([[0.0000, 0.0000, 0.0000, 0.0191, 0.1118]], grad_fn=<ReluBackward0>)
loss : tensor(1.5247, grad_fn=<NllLossBackward0>)
Finished training in 65.125685 sec
Total Loss :  1.5247398614883423


In [209]:
outputs2 = None
outputs = None
for i, data in enumerate(test_loader):
    inputs, labels = data['image'], data['label'].view(1,-1)
    inputs.requires_grad = True
    # # display(inputs)
    # start = time.time()
    # outputs2 = model2(inputs)
    # print(outputs2.size())
    # delta = time.time() - start
    # print(f"Finished training in {delta:0.6f} sec")
    # # display(outputs2, True)

    start = time.time()
    outputs = model(inputs)
    print(outputs.size())
    delta = time.time() - start
    print(f"Finished training in {delta:0.6f} sec")
    # display(outputs)
    break

torch.Size([1, 3, 223, 223])
Finished training in 0.007386 sec
[c] : 0
torch.Size([1, 3, 223, 223])
Finished training in 119.824546 sec


In [210]:
# model.zero_grad()
# # loss = model.backward(outputs2)
# loss = outputs2 - outputs  # Replace with your actual loss computation
# # loss_scalar = loss.mean()
# print(loss)
# loss.backward()
# # loss.backward()

tensor([[[[-0.7054, -0.6054, -0.7054,  ..., -0.6054, -0.8054, -0.6054],
          [-0.6054, -0.7054, -0.4054,  ..., -0.6054, -0.3054, -0.6054],
          [-0.7054, -0.7054, -0.6054,  ..., -0.8054, -0.7054, -0.5054],
          ...,
          [-0.7054, -0.4054, -0.7054,  ..., -0.6054, -0.6054, -0.5054],
          [-0.4054, -0.6054, -0.5054,  ..., -0.5054, -0.5054, -0.6054],
          [-0.7054, -0.6054, -0.8054,  ..., -0.7054, -0.6054, -0.7054]],

         [[-0.2247, -0.4247, -0.2247,  ..., -0.2247, -0.0247, -0.3247],
          [-0.4247, -0.2247, -0.1247,  ..., -0.2247, -0.3247, -0.2247],
          [-0.3247, -0.3247, -0.1247,  ..., -0.2247, -0.1247, -0.3247],
          ...,
          [-0.2247, -0.2247, -0.3247,  ..., -0.3247, -0.1247, -0.1247],
          [-0.1247, -0.4247, -0.3247,  ..., -0.0247, -0.1247, -0.2247],
          [-0.0247, -0.3247, -0.2247,  ..., -0.1247, -0.4247, -0.3247]],

         [[-0.4891, -0.4891, -0.3891,  ..., -0.6891, -0.2891, -0.3891],
          [-0.3891, -0.4891, -

RuntimeError: grad can be implicitly created only for scalar outputs

In [181]:
import torch.optim as optim

In [212]:
# define two tensors
A = torch.tensor(2., requires_grad=True)
print("Tensor-A:", A)
B = torch.tensor(5., requires_grad=True)
print("Tensor-B:", B)
  
# define a function using above defined
# tensors
optimizer = optim.SGD([A,B], lr=0.5)
for i in range(10):
    optimizer.zero_grad()
    loss = B - A
    print("loss :", loss)
    # call the backward method
    loss.backward()
    optimizer.step()
    print("A:grad", A.grad, A)
    print("B.grad:", B.grad, B)
    print()
# print the gradients using .grad

Tensor-A: tensor(2., requires_grad=True)
Tensor-B: tensor(5., requires_grad=True)
loss : tensor(3., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(2.5000, requires_grad=True)
B.grad: tensor(1.) tensor(4.5000, requires_grad=True)

loss : tensor(2., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(3., requires_grad=True)
B.grad: tensor(1.) tensor(4., requires_grad=True)

loss : tensor(1., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(3.5000, requires_grad=True)
B.grad: tensor(1.) tensor(3.5000, requires_grad=True)

loss : tensor(0., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(4., requires_grad=True)
B.grad: tensor(1.) tensor(3., requires_grad=True)

loss : tensor(-1., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(4.5000, requires_grad=True)
B.grad: tensor(1.) tensor(2.5000, requires_grad=True)

loss : tensor(-2., grad_fn=<SubBackward0>)
A:grad tensor(-1.) tensor(5., requires_grad=True)
B.grad: tensor(1.) tensor(2., requires_grad=True)

loss : tensor(-3., grad_fn=<SubBac